In [1]:
%pip install ultralytics

%pip install opencv-python-headless matplotlib

print("Installation complete.")

You should consider upgrading via the '/Users/mariaplatek/projects/AXION/YOLOv11/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/mariaplatek/projects/AXION/YOLOv11/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Installation complete.


In [19]:
import torch
from ultralytics import YOLO
import os
import glob
import numpy as np

In [3]:
if torch.cuda.is_available():
    print("NVIDIA GPU (CUDA) detected.")
    device = "cuda:0"
elif torch.backends.mps.is_available():
    print("Apple Silicon (MPS) detected!")
    device = "mps"
else:
    print("Using CPU.")
    device = "cpu"

Apple Silicon (MPS) detected!


In [ ]:
best_model_path = "weights/best.pt"
dataset_yaml = "dataset/data.yaml"
test_images_dir = "dataset/images/test"
test_labels_dir = "dataset/labels/test"

In [16]:
if os.path.exists(best_model_path):
    model = YOLO(best_model_path)
    print(f"Model loaded successfully from {best_model_path}")
else:
    raise FileNotFoundError(f"Could not find model at {best_model_path}. Check your training output folder.")

Model loaded successfully from yolo26m_v3_512/weights/best.pt


In [ ]:
print(f"\nRunning Evaluation on Test Set using {device}...")
metrics = model.val(
    data=dataset_yaml, 
    split='test', 
    device=device,
    plots=True
)
f1_mean = np.mean(metrics.box.f1)

print("\n--- FINAL METRICS ---")
print(f"mAP@50:    {metrics.box.map50:.2%}")
print(f"mAP@50-95: {metrics.box.map:.2%}")
print(f"Precision: {metrics.box.mp:.2%}")
print(f"Recall:    {metrics.box.mr:.2%}")
print(f"F1 Score: {f1_mean:.2%}")



Running Evaluation on Test Set using mps...
Ultralytics 8.4.14 🚀 Python-3.9.6 torch-2.8.0 MPS (Apple M4 Pro)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 323.1±75.2 MB/s, size: 101.0 KB)
val: Scanning /Users/mariaplatek/projects/AXION/YOLOv11/dataset/labels/test.cache... 1968 images, 1417 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1968/1968 1.0Git/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 123/123 3.2it/s 38.4s0.3s
                   all       1968       1427      0.673      0.693       0.71      0.362
Speed: 0.1ms preprocess, 14.2ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /Users/mariaplatek/projects/AXION/YOLOv11/runs/detect/val11

--- FINAL METRICS ---
mAP@50:    70.98%
mAP@50-95: 36.19%
Precision: 67.27%
Recall:    69.31%
F1 Score: 68.27%


In [29]:
# --- Dynamic Prediction ---
model.to(device)

# Grab the first available image in the test directory dynamically
test_images = glob.glob(os.path.join(test_images_dir, "*.png"))

if test_images:
    sample_image = test_images[0]
    print(f"\nRunning prediction on sample: {sample_image}")
    
    results = model.predict(
        sample_image, 
        conf=0.1, #change to higher when trained model
        device=device,
        half=False
    )

    for r in results:
        boxes = r.boxes
        print("\n--- RAW MODEL OUTPUT ---")
        print(f"Coordinates (xyxy): {boxes.xyxy.tolist()}") 
        print(f"Confidence scores: {boxes.conf.tolist()}")
        print(f"Class IDs: {boxes.cls.tolist()}")
else:
    print(f"Warning: No .png images found in {test_images_dir}!")


Running prediction on sample: dataset/images/test/cadica_p37_v1_p37_v1_00022.png

image 1/1 /Users/mariaplatek/projects/AXION/YOLOv11/dataset/images/test/cadica_p37_v1_p37_v1_00022.png: 512x512 1 stenosis, 14.8ms
Speed: 1.0ms preprocess, 14.8ms inference, 0.9ms postprocess per image at shape (1, 3, 512, 512)

--- RAW MODEL OUTPUT ---
Coordinates (xyxy): [[231.77587890625, 127.27021026611328, 310.22900390625, 174.7377471923828]]
Confidence scores: [0.4400663375854492]
Class IDs: [0.0]


In [30]:
def calculate_single_iou_by_area(boxA, boxB):
    xA, yA = max(boxA[0], boxB[0]), max(boxA[1], boxB[1])
    xB, yB = min(boxA[2], boxB[2]), min(boxA[3], boxB[3])

    # 1. Calculate TP (True Positive area) -> The Intersection
    TP = max(0, xB - xA) * max(0, yB - yA)
    
    # Calculate the total area of each individual box
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1]) # Predicted Box
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1]) # Ground Truth Box
    
    # Calculate FP (False Positive area) -> Predicted area minus Intersection
    FP = boxAArea - TP
    
    # Calculate FN (False Negative area) -> Ground Truth area minus Intersection
    FN = boxBArea - TP
    
    # IoU = TP / (TP + FP + FN)
    denominator = TP + FP + FN
    
    if denominator == 0:
        return 0.0
        
    return TP / denominator

def get_average_iou(model, test_images_dir, test_labels_dir):
    print("\nCalculating Average IoU using (TP / TP+FP+FN)...")
    total_iou = 0
    true_positives = 0

    image_paths = []
    for ext in ('*.jpg', '*.jpeg', '*.png'):
        image_paths.extend(glob.glob(os.path.join(test_images_dir, ext)))

    for img_path in image_paths:
        # prediction
        results = model(img_path, device=device, verbose=False)
        pred_boxes = results[0].boxes.xyxy.cpu().numpy()
        img_h, img_w = results[0].orig_shape 
        
        filename = os.path.basename(img_path)
        label_filename = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(test_labels_dir, label_filename)
        
        if not os.path.exists(label_path):
            continue 
            
        gt_boxes = []
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    _, cx_n, cy_n, w_n, h_n = map(float, parts[:5])
                    cx, cy = cx_n * img_w, cy_n * img_h
                    w, h = w_n * img_w, h_n * img_h
                    x1, y1 = cx - w/2, cy - h/2
                    x2, y2 = cx + w/2, cy + h/2
                    gt_boxes.append([x1, y1, x2, y2])
        
        for p_box in pred_boxes:
            best_iou = 0
            for g_box in gt_boxes:
                # Use the new explicit TP/FP/FN function
                iou = calculate_single_iou_by_area(p_box, g_box)
                if iou > best_iou:
                    best_iou = iou
            
            # If overlap is 50% or higher, count it as a True Positive
            if best_iou >= 0.50:
                total_iou += best_iou
                true_positives += 1

    if true_positives > 0:
        avg_iou = total_iou / true_positives
        print(f"\n--> Evaluated {true_positives} True Positive detections.")
        print(f"--> Average Box Overlap (IoU): {avg_iou*100:.2f}%")
    else:
        print("\n--> No valid detections found to calculate IoU.")

get_average_iou(model, test_images_dir, test_labels_dir)


Calculating Average IoU using (TP / TP+FP+FN)...

--> Evaluated 1153 True Positive detections.
--> Average Box Overlap (IoU): 70.59%
